In [2]:
import torch
from torch.nn import CrossEntropyLoss, BCELoss, BCEWithLogitsLoss
from torch_geometric.data import DataLoader
import numpy as np
from model import GraphEncoder
from readData import LoadData, LoadGraph
import os
from MultiCostNetworks import MultiCostNetworks
import sys

import torch.nn.functional as F
import time

print("Load necessary libs .....")

Load necessary libs .....


In [9]:
# gDB = "C9_NY_1K"
# path = "/home/gqxwolf/shared_git/BackboneIndex/Data/"+gDB+"/"
# path
path = '/Users/hyingchen/PycharmProjects/ICDE23/GNNquery/Graph_data/datasets/sample_graph/'

In [10]:
graph = LoadGraph(path)
graph

(5000, Data(x=[5000, 2], edge_index=[2, 14194], edge_attr=[14194, 3]))

In [11]:
node_dim = 2
edge_dim = 3
embedding_dim = 512
hidden_dim = 512
batch_size = 64
enable_embed = True
output_dim =3
model_name="SAGE"
n=1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
# train_paths_folder = "/home/gqxwolf/shared_git/BackboneIndex/Data/results/"+gDB+"/training_GNN"
# root = "/home/gqxwolf/mydata/GNN_Combinatroal/data/"+gDB
train_paths_folder = "/Users/hyingchen/PycharmProjects/ICDE23/GNNquery/Graph_data/datasets/sample_graph/"
root = "/Users/hyingchen/PycharmProjects/ICDE23/GNNquery/Graph_data/datasets/sample_graph"

train_dataset = MultiCostNetworks(graph, train_paths_folder = train_paths_folder, root=root, split='train')
test_dataset = MultiCostNetworks(graph, train_paths_folder = train_paths_folder, root=root,split='test')

Processing...


cpu
Process (5000, Data(x=[5000, 2], edge_index=[2, 14194], edge_attr=[14194, 3]))


0it [00:00, ?it/s]


ValueError: invalid literal for int() with base 10: 'SegInfo.txt'

In [5]:
train_loader = DataLoader(train_dataset, drop_last=True, batch_size=batch_size)

for data in train_loader:
  print(data)
  src_list = torch.tensor([data.src_node_idx[i].item() + i * n for i in range(len(data.src_node_idx))])
  dest_list = torch.tensor([data.dest_node_idx[i].item() + i * n for i in range(len(data.dest_node_idx))])
  print(src_list)
  break

Batch(batch=[64000], dest_node_idx=[64], edge_attr=[195712, 3], edge_index=[2, 195712], ptr=[65], src_node_idx=[64], x=[64000, 2], y=[64000])
tensor([  225,  1845,  2590,  3530,  4975,  5146,  6053,  7129,  8327,  9160,
        10867, 11829, 12671, 13519, 14945, 15337, 16097, 17321, 18187, 19608,
        20817, 21499, 22051, 23226, 24888, 25613, 26682, 27973, 28224, 29374,
        30960, 31815, 32083, 33482, 34721, 35812, 36008, 37558, 38533, 39709,
        40747, 41374, 42492, 43534, 44006, 45750, 46155, 47232, 48875, 49347,
        50084, 51961, 52012, 53234, 54339, 55742, 56907, 57740, 58090, 59566,
        60885, 61782, 62271, 63589])


In [6]:
train_loader = DataLoader(train_dataset, drop_last=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, drop_last=True, batch_size=batch_size)

model = GraphEncoder(node_dim, edge_dim, embedding_dim, hidden_dim, output_num_class=output_dim, device=device, enable_embedding=enable_embed,
                     batch=batch_size, model_name = model_name).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)

64
512
Finish model initialization !!!!


In [7]:
def train():
    model.train()

    total_loss = 0
    total_pre_n = 0
    total_n = 0
    total_graphs = 0

    for index, data in enumerate(train_loader):
        start_time = time.time()
        data = data.to(device)
        # print(data, data.src_node_idx, data.dest_node_idx)
        logits = model(data)
        # print(logits)
        # print(data.y, (data.y.cpu().detach().numpy() == 1).sum())
        loss = F.nll_loss(logits, data.y)

        total_pre_n += (np.argmax(logits.cpu().detach().numpy(), axis=1) == 1).sum() + (
                    np.argmax(logits.cpu().detach().numpy(), axis=1) == 2).sum()
        total_n += ((data.y.cpu().detach().numpy() == 1).sum())

        total_graphs += data.num_graphs
        total_loss += loss.item() * data.num_graphs

        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()
        # print(index, time.time()-start_time)

    return total_loss / len(train_dataset), total_pre_n / total_graphs, total_n / total_graphs

print("inital the train function")

inital the train function


In [9]:
@torch.no_grad()
def test(loader):
    model.eval()

    total_loss = 0
    total_pre_n = 0
    total_n = 0
    total_graphs = 0

    for data in loader:
        data = data.to(device)
        logits = model(data)

        loss = F.nll_loss(logits, data.y)

        total_pre_n += (np.argmax(logits.cpu().detach().numpy(), axis=1) == 1).sum() + (
                np.argmax(logits.cpu().detach().numpy(), axis=1) == 2).sum()
        total_n += ((data.y.cpu().detach().numpy() == 1).sum())

        total_graphs += data.num_graphs

        total_loss += loss.item() * data.num_graphs

        # if n!=0:
        #     print(count, np.argmax(logits.cpu().numpy(),axis=1), n )
        #     sys.exit()

    return total_loss / len(loader), total_pre_n / total_graphs, total_n / total_graphs
print("initial the test function")

initial the test function


In [8]:
best_val_auc = test_auc = 0
for epoch in range(0, 10000):
    start_time = time.time()
    loss, pre_n, n = train()
    val_auc, test_p_n, test_n = test(test_loader)
    end_time = time.time()
    if epoch%10==0:
      print("Epoch:{}, Loss:{:.4f}[{:.5f} - {:.5f}], Val:{:.4f}[{:.5f} - {:.5f}], running time each epoch:{:.2f}".format(epoch, loss, pre_n, n, val_auc, test_p_n, test_n, (end_time - start_time)))
    if epoch%200==0:
      check_pt_path = "/content/drive/MyDrive/ColabNotebooks/GNN_Skyline/models/{}/{}_{}_{}_{}_train_checkpoint_{}_2GNN3FF_10k_train_samples.pt".format(gDB, embedding_dim, hidden_dim, batch_size, epoch,model_name)
      torch.save({'epoch': epoch,'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, check_pt_path)


torch.Size([64000, 2])
torch.Size([64000, 1536])


SystemExit: 0